1. 安装最新版本的modelscope和swift

In [ ]:
!pip install modelscope ms-swift -U
!pip install tf-keras==2.16.0 --no-dependencies

2. 加载数据集

In [1]:
from modelscope import MsDataset
dataset = MsDataset.load('swift/classical_chinese_translate')
dataset = dataset['train']

3. 查看数据集内容

In [ ]:
print(dataset)
print(dataset['conversations'][0])

4. 加载模型、分词器、模板

In [ ]:
import torch
from modelscope import AutoModelForCausalLM
from modelscope import AutoTokenizer
from swift.llm import get_template, TemplateType
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-7B-Instruct', trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-7B-Instruct', torch_dtype=torch.bfloat16, device_map='auto', trust_remote_code=True)
print(model)
print(tokenizer('I like AI'))


template = get_template(TemplateType.qwen2_5, tokenizer, max_length=400)
ret = template.encode({'query': 'what is your hobby?', 'response': 'I like AI'})
print(ret)
tokenizer.decode(ret[0]['input_ids'])

5. 预处理数据集

In [ ]:
from swift.llm.utils.preprocess import ConversationsPreprocessor
ds = ConversationsPreprocessor()(dataset)
print(ds[0])

def encode(example):
    example, kwargs = template.encode(example)
    if 'input_ids' not in example:
        return {
            'input_ids': None,
            'labels': None,
        }
    return example

ds = ds.select(range(300)).map(encode).filter(lambda e: e.get('input_ids'))
ds = ds.train_test_split(test_size=0.1)

train_dataset, val_dataset = ds['train'], ds['test']
print('===========================================')
print(train_dataset[0])

6. 加载LoRA

In [6]:
from swift import Swift, LoraConfig


lora_config = LoraConfig(
                r=8,
                target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
                lora_alpha=32,
                lora_dropout=0.05)
model = Swift.prepare_model(model, lora_config)

7. 训练

In [ ]:
# A100 18G memory
from swift import Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch


train_args = Seq2SeqTrainingArguments(
    output_dir='output',
    learning_rate=1e-4,
    num_train_epochs=1,
    eval_steps=5,
    save_steps=5,
    evaluation_strategy='no',
    save_strategy='steps',
    dataloader_num_workers=4,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    logging_steps=2,
)

print(train_dataset[0])

trainer = Seq2SeqTrainer(
    model=model,
    args=train_args,
    data_collator=template.data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer)

trainer.train()

8. 看看存了什么

In [ ]:
!ls output/checkpoint-11

9. 推理

In [ ]:
from modelscope import GenerationConfig
query = '你喜欢什么'
inputs, kwargs = template.encode({'query': query})
print(inputs)
generation_config = GenerationConfig(max_new_tokens=512, top_p=0.7, temperature=0.3)
inputs['input_ids'] = torch.tensor(inputs['input_ids'])[None].cuda()
generate_ids = model.generate(generation_config=generation_config, **inputs)
print(generate_ids)
print(tokenizer.decode(generate_ids[0]))

10. 界面

In [ ]:
!pip install gradio==3.50.2
!swift web-ui